## ReMKiT1D input generator - Calculation tree matrix model generation example

This notebook showcases an example of using the addNodeMatrixTermModel function from common_models (added in v1.1.0).

In [ ]:
import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid ,Node ,treeDerivation
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.common_models as cm
import RMK_support.dashboard_support as ds

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt
import panel as pn

### Wrapper initialization


In [ ]:
rk = RKWrapper()

### Global parameters for writing the files


In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_calc_tree_model_test/"
rk.setHDF5Path(hdf5Filepath) # The input and output location of any HDF5 files used/generated by the code

### Setting options for external libraries used by ReMKiT1D


#### MPI


In [ ]:
numProcsX = 4 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic direction 
haloWidth = 1 # Halo width in cells 
numProcs = numProcsH * numProcsX
rk.setMPIData(numProcsX,numProcsH,haloWidth)

### Normalization


In [ ]:
rk.setNormDensity(1.0e19) #n_0
rk.setNormTemperature(10.0) #T_0
rk.setNormRefZ(1.0) # reference ion charge for e-i collision time

### Grid initialization


In [ ]:
# In normalized length or in meters - defaults to normalized unless isLengthInMeters=True in Grid
xGridWidths = 0.025*np.ones(512)
# In normalized velocity - default normalization is thermal velocity sqrt(m_e * k * T_e/2)
vGrid = np.ones(1) 
lMax = 0
gridObj = Grid(xGridWidths, vGrid, lMax, interpretXGridAsWidths=True)

rk.grid = gridObj

### Variable container


#### Adding variables


In [ ]:
T = 1 + np.exp(-(gridObj.xGrid-np.mean(gridObj.xGrid))**2) # A Gaussian perturbation

# These will add both the variable 'v' and 'v_dual'
rk.addVarAndDual('T',T,isCommunicated=True) 
rk.addVarAndDual('q',isCommunicated=True,primaryOnDualGrid=True)
rk.addVar('time',isDerived=True,isScalar=True)

### Models 

Nonlinear temperature conduction model

$$\frac{\partial T}{\partial t} + \nabla q = 0$$
$$q = -0.1T^{5/2}\nabla T$$


In [ ]:
newModel = sc.CustomModel(modelTag="divq")

# heat flux divergence
divFluxTerm = sc.GeneralMatrixTerm(evolvedVar='T',implicitVar='q_dual',customNormConst=-1.0,stencilData=sc.staggeredDivStencil())
newModel.addTerm("divFlux",divFluxTerm)

#Identity term for the heat flux dual component
qIdentity = sc.GeneralMatrixTerm(evolvedVar='q_dual',customNormConst=-1.0,stencilData=sc.diagonalStencil())
newModel.addTerm("q_identity",qIdentity)
rk.addModel(newModel)

The heat flux here lives on the staggered grid, so the row variable node should also be on the dual grid, hence the use of T_dual



In [ ]:
cm.addNodeMatrixTermModel(rk,"cond","q_dual",[(-0.1*Node("T_dual")**(5/2),"T")],[sc.staggeredGradStencil()]) # represents the RHS of the second equation of the model

### Integrator options


In [ ]:
# the implicit BDE integrator that checks convergence based on the variables 'n' and 'u_dual'
integrator = sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=['T','q_dual']) 

rk.addIntegrator("BE",integrator)

#### Global integrator options and timestep control

In [ ]:
# fixed timestep in this example
initialTimestep=0.1 # in normalized time units
rk.setIntegratorGlobalData(1, # number of allowed implicit term groups - grouping everything into one group per model
                           1, # number of allowed general term groups
                           initialTimestep) 

#### Controlling integration steps

In [ ]:
# a single integration step evolving all models
bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

### Time loop options

In [ ]:
rk.setFixedNumTimesteps(10000)
rk.setFixedStepOutput(200)

### Create config file

In [ ]:
rk.writeConfigFile()

### Set global plotting options

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

### Load data from ReMKiT1D output files

In [ ]:
numFiles = 50
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

In [ ]:
"T" in rk.varCont.dataset.data_vars.keys()

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show()